In [ ]:
import keras 
from keras import backend as k 
from keras.engine import Input, Model 
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from keras.optimizers import Adam
from keras.layers.merge import concatenate
k.set_image_data_format("channels_first")

In [ ]:
# deconvolution3d changed to conv3dtranspose 
# the depth of u-net = the number of down convolution 
# the shape of input layer is (num_channels,height,width,length)
# ? is batch_size (batch_size, num_channels, height, width, length)
# for this model we gonna use 2 depth

In [ ]:
input_layer = Input(shape=(4,160,160,16))
input_layer

<KerasTensor: shape=(None, 4, 160, 160, 16) dtype=float32 (created by layer 'input_1')>

In [ ]:
#downward path(contracting)
# the numbers of filters for depth 0 
# fillters(i) = 32 * 2 to the power i
# where i is the current depth
#so at depth i = 0 filters = 32
# in downward path The (height, width, length) of the input gets smaller as you move down this path, and the number of channels increases

In [ ]:
down_depth_0_layer_0 = Conv3D(filters=32,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1))(input_layer)
down_depth_0_layer_0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'conv3d')>

In [ ]:
down_depth_0_layer_0 = Activation("relu")(down_depth_0_layer_0)
down_depth_0_layer_0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'activation')>

In [ ]:
# depth 0 layer 1 
# formula for calculating filters 
# filters(i) = 32 * 2 to the power 0 * 2  

In [ ]:
down_depth_0_layer_1 = Conv3D(filters=64,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1))(down_depth_0_layer_0)
down_depth_0_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'conv3d_2')>

In [ ]:
down_depth_0_layer_1 = Activation("relu")(down_depth_0_layer_1)
down_depth_0_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_2')>

In [ ]:
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool 

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'max_pooling3d_1')>

In [ ]:
# depth 1 layer 0 
# the number of filters 32 * 2 to the power 1 = 64

In [ ]:
down_depth_1_layer_0 = Conv3D(filters=64,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1))(down_depth_0_layer_pool)
down_depth_1_layer_0

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'conv3d_4')>

In [ ]:
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'activation_4')>

In [ ]:
#depth 1 layer 1 
# filters = 32 * 2 to the power 1 * 2 = 128

In [ ]:
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<KerasTensor: shape=(None, 128, 80, 80, 8) dtype=float32 (created by layer 'activation_5')>

In [ ]:
# expanding upward path
# depth 0 layer 0

In [ ]:
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<KerasTensor: shape=(None, 128, 160, 160, 16) dtype=float32 (created by layer 'up_sampling3d')>

In [ ]:
print(up_depth_0_layer_0)
print()
print(down_depth_0_layer_1)

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 160, 160, 16), dtype=tf.float32, name=None), name='up_sampling3d/concat_2:0', description="created by layer 'up_sampling3d'")

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 160, 160, 16), dtype=tf.float32, name=None), name='activation_2/Relu:0', description="created by layer 'activation_2'")


In [ ]:
up_depth_1_concat = concatenate([up_depth_0_layer_0,down_depth_0_layer_1],axis=1)
up_depth_1_concat

<KerasTensor: shape=(None, 192, 160, 160, 16) dtype=float32 (created by layer 'concatenate')>

In [ ]:
up_depth_1_layer_1 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_6')>

In [ ]:
up_depth_1_layer_2 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_7')>

In [ ]:
final_conv = Conv3D(filters=3,kernel_size=(1,1,1),padding="valid",strides=(1,1,1))(up_depth_1_layer_2)

In [ ]:
final_activation = Activation('sigmoid')(final_conv)
final_activation

<KerasTensor: shape=(None, 3, 160, 160, 16) dtype=float32 (created by layer 'activation_9')>

In [ ]:
model = Model(inputs=input_layer,outputs=final_activation)
model.compile(optimizer=Adam(lr=0.00001),
              loss="categorical_crossentropy",metrics=['categorical_accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4, 160, 160, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 160, 160, 0           conv3d[0][0]                     
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 160, 160, 55360       activation[0][0]                 
______________________________________________________________________________________________